# Kaggle資料集下載與清理

資料來源：https://www.kaggle.com/datasets/fronkongames/steam-games-dataset



*   本資料集csv檔有誤，所以拿的是json檔自己讀取重做csv



In [ ]:
json_path = "games.json"
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

records = []
for appid, info in data.items():
    record = {"appid": appid}  # 對齊欄位名
    record.update(info)
    records.append(record)

df2 = pd.DataFrame(records)

In [ ]:
df2.info()

In [ ]:
missing_info = df2.isnull().sum().reset_index()
missing_info.columns = ['Column', 'Missing_Count']
print(missing_info.to_markdown(index=False, floatfmt=".2f"))

In [ ]:
import re
def clean_date(date_str):
    if pd.isna(date_str) or date_str == '':
        return date_str

    date_str = str(date_str).strip()

    # Try parsing standard formats like "Jul 5, 2024" or "2024-07-05"
    try:
        dt = pd.to_datetime(date_str, errors='raise')
        return dt.strftime('%Y-%m-%d')
    except:
        pass

    # If parsing failed, look for a 4-digit year
    match = re.search(r'(\d{4})', date_str)
    if match:
        return match.group(1)

    return date_str

In [ ]:
# 通用進度條裝飾器 / 包裝器
# 因為呼叫clean_date要一段時間
import functools
import sys

def progress_bar(func=None, *, total=None, width=30, show_result=False):
    """
    通用進度條裝飾器，支援多元素函數。
    - func: 要執行的函數，每次處理一個元素或多個欄位
    - total: 總數量 (若不指定會自動用 len(iterable))
    - width: 進度條寬度
    - show_result: 是否回傳每次 func 的結果列表
    """
    if func is None:
        return lambda f: progress_bar(f, total=total, width=width, show_result=show_result)

    @functools.wraps(func)
    def wrapper(iterable, *args, **kwargs):
        n = len(iterable) if total is None else total
        results = []

        for i, item in enumerate(iterable, 1):
            # 如果 item 是 tuple/list 就拆開傳給 func
            if isinstance(item, (tuple, list)):
                res = func(*item, *args, **kwargs)
            elif isinstance(item, dict):
                res = func(**item, **kwargs)
            else:
                res = func(item, *args, **kwargs)

            if show_result:
                results.append(res)

            # 更新進度條
            progress = int(width * i / n)
            bar = '#' * progress + '-' * (width - progress)
            percent = i / n * 100
            print(f"\r[{bar}] {percent:6.2f}% ({i}/{n})", end='')
            sys.stdout.flush()

        print()  # 換行
        if show_result:
            return results

    return wrapper

df2['price'] = pd.to_numeric(df2['price'], errors='coerce').fillna(0)
df2['Free/paid'] = df2['price'].apply(lambda x: 0 if x == 0 else 1)
df2['release_date'] = pd.to_datetime(df2['release_date'])
df2['release_year'] = df2['release_date'].dt.year
df2['release_date'] = progress_bar(lambda x: clean_date(x), show_result=True)(df2['release_date'])

In [ ]:
# 此分析針對 2021 ~ 2025 的資料
df2 = df2[(df2['release_year'] >= 2021) & (df2['release_year'] <= 2025)]
df2 = df2[['appid','name','release_date', 'price', 'genres', 'categories',
          'dlc_count', 'metacritic_score','recommendations','developers','publishers',
          'user_score', 'average_playtime_forever', 'average_playtime_2weeks',
          'median_playtime_forever', 'median_playtime_2weeks',
          'discount', 'peak_ccu', 'Free/paid', 'release_year'
          ]]
df2.info()

In [ ]:
# 填空值並轉字串
df2['genres'] = df2['genres'].fillna('').astype(str)
df2['categories'] = df2['categories'].fillna('').astype(str)

# 將 list 字串轉成分號分隔字串
def normalize_multi(x):

    x = x.strip()
    if x.startswith('[') and x.endswith(']'):   # 例如 "['Action', 'Indie']"
        x = x[1:-1]                             # 去掉 []
        x = x.replace("'", "").replace('"', '') # 去掉引號
        x = x.replace(", ", ";")               # 改成分號分隔
    return x

df2['genres'] = df2['genres'].apply(normalize_multi)
df2['categories'] = df2['categories'].apply(normalize_multi)

# 先備份原始欄位
df2['genres_orig'] = df2['genres']
df2['categories_orig'] = df2['categories']

# 清理欄位
df2['genres_clean'] = df2['genres'].fillna('').astype(str).apply(normalize_multi)
df2['categories_clean'] = df2['categories'].fillna('').astype(str).apply(normalize_multi)

# 找出被清理掉變成空值或 NaN 的資料
genres_dropped = df2[df2['genres_clean'].isna() | (df2['genres_clean'] == '')]['genres_orig']
categories_dropped = df2[df2['categories_clean'].isna() | (df2['categories_clean'] == '')]['categories_orig']

# 顯示結果
print("被清理掉的 genres：")
print(genres_dropped.value_counts())

print("\n被清理掉的 categories：")
print(categories_dropped.value_counts())

# genres
genres_dummies = (
    pd.get_dummies(df2['genres'].str.split(';').explode(), prefix='genres', dtype='uint8')
    .groupby(level=0).sum()
    .astype(pd.SparseDtype("uint8", 0))
)

# categories
categories_dummies = (
    pd.get_dummies(df2['categories'].str.split(';').explode(), prefix='categories', dtype='uint8')
    .groupby(level=0).sum()
    .astype(pd.SparseDtype("uint8", 0))
)

# --- 4. 合併回 df ---
df2 = pd.concat([df2, genres_dummies, categories_dummies], axis=1)

In [ ]:
# 將 NaN 或空字串統一處理成 NaN（方便刪除）
df2['genres'] = df2['genres'].replace('', pd.NA)
df2['categories'] = df2['categories'].replace('', pd.NA)

# 刪除 genres 或 category 為 NaN 的列
df_cleaned = df2.dropna(subset=['genres', 'categories'])

# 檢查結果
print(f"原資料筆數: {len(df2)}")
print(f"清理後筆數: {len(df_cleaned)}")
print(f"被刪除筆數: {len(df2) - len(df_cleaned)}")

In [ ]:
# 過濾欄位，排除 genre_ 或 category_ 開頭的欄位
cols_to_show = [c for c in df2.columns if not (c.startswith('genres_') or c.startswith('categories_'))]

# 顯示欄位資訊(排除拆開後的genre和category)
df_cleaned[cols_to_show].info()

df_cleaned.to_csv('steam_games_all_clean.csv', encoding='utf-8', index=False)

# 下載 Indie Games Award(IGA) 得獎作品名單

In [ ]:
# 使用html讀取整理
# 官網每年的格式都不同，只能分開處理

import urllib.request as req
import bs4
import pandas as pd

year_prizes = []

year = 25
url = "https://tgs.tca.org.tw/indie_award_winners.php?b=e&a="+str(year)
response = req.urlopen(url)
html = bs4.BeautifulSoup(response)

header = []
rows = html.find_all("tr")

header = [t.text.strip() for t in html.find_all("th")]
if not header and rows:
    sample_cells = rows[1].find_all("td", {"align": "center"})
    if len(sample_cells) == 4:
        header = ["Category", "Country", "Winner", "Studio"]
    else:
        header = [td.text.strip() for td in rows[0].find_all("td")]

for row in rows[1:]:
    cells = row.find_all("td", {"align": "center"})
    if len(cells) == len(header):
        content = cells[0].text.strip()
        prize_dict = {header[i]: cells[i].text.strip() for i in range(len(header))}
        prize_dict["Year"] = 2000 + year
        year_prizes.append(prize_dict)

df = pd.DataFrame(year_prizes)
df.to_csv('Indie_' + str(2000+year))



In [ ]:
# 23年可能網頁格式有改，另外抓
year_prizes = []

year = 23
url = "https://tgs.tca.org.tw/indie_award_winners.php?b=e&a=23"
request = req.Request(url)
response = req.urlopen(request)
html = bs4.BeautifulSoup(response, "html.parser")

# 1. 直接找所有的 <td>
all_tds = html.find_all("td")

for i, td in enumerate(all_tds):
    # 2. 清理文字中的任意空白 (\s+)
    text = re.sub(r'\s+', ' ', td.text).strip()

    # 3. 如果這格是獎項名稱 (包含 Best 或 Grand)
    if "Best" in text or "Grand" in text:
        # 排除掉長度太長的介紹文字
        if len(text) < 50:
            try:
                # 2023 年這行有 4 欄，所以獎項後面的 3 格就是我們要的
                prize_dict = {
                    "Category": text,
                    "Country": re.sub(r'\s+', ' ', all_tds[i+1].text).strip(),
                    "Winner": re.sub(r'\s+', ' ', all_tds[i+2].text).strip(),
                    "Studio": re.sub(r'\s+', ' ', all_tds[i+3].text).strip(),
                    "Year": 2000 + year
                }
                year_prizes.append(prize_dict)
            except IndexError:
                continue

df = pd.DataFrame(year_prizes)
df = df.drop_duplicates(subset=['Category', 'Winner'])
df = df[df['Winner'].notna() & (df['Winner'] != "")]
df.to_csv('Indie_2023')

In [ ]:
dfs = []

# 2020
df_2020 = pd.read_csv("/content/Indie_2020", encoding="utf-8", index_col=0)
df_2020 = df_2020.rename(columns={'PRIZE': 'Award Category', 'Studio': 'Developer/Team', 'GAME TITLE':'Game Title'})
# 拆分 Studio 與 Country Area
df_2020['Country Area'] = df_2020['Developer/Team'].str.split('(').str[1].str.replace(')', '', regex=False)
df_2020['Developer/Team'] = df_2020['Developer/Team'].str.split('(').str[0].str.strip()
dfs.append(df_2020)

# 2021
df_2021 = pd.read_csv("/content/Indie_2021", encoding="utf-8", index_col=0)
df_2021 = df_2021.rename(columns={'PRIZE': 'Award Category', 'GAME TITLE':'Game Title'})
dfs.append(df_2021)

# 2022
df_2022 = pd.read_csv("/content/Indie_2022", encoding="utf-8", index_col=0)
df_2022 = df_2022.rename(columns={'Prize': 'Award Category'})
dfs.append(df_2022)

# 2023
df_2023 = pd.read_csv("/content/Indie_2023", encoding="utf-8", index_col=0)
df_2023 = df_2023.rename(columns={'Category': 'Award Category', 'Country': 'Country Area', 'Winner':'Game Title', 'Studio': 'Developer/Team'})
dfs.append(df_2023)

# 2024 & 2025
# 注意：原程式碼這邊如果都賦值給 df 會導致 2024 被 2025 蓋掉，這裡修正為分別讀取
dfs.append(pd.read_csv("/content/Indie_2024", encoding="utf-8", index_col=0))
dfs.append(pd.read_csv("/content/Indie_2025", encoding="utf-8", index_col=0))

# 合併並存成一個最終檔案
df_all = pd.concat(dfs, axis=0, ignore_index=True)
df_all.to_csv("Indie_Game_Awards_2020_2025.csv", encoding="utf-8", index=False)

# 資料清理

In [ ]:
# 折扣相關計算
df['discount(%)'] = df['discount'] / 100

# 遊玩時間
df['average_playtime_change'] = np.where(
    df['average_playtime_forever'] > 0,
    (df['average_playtime_2weeks'] - df['average_playtime_forever'])
    / df['average_playtime_forever'],
    np.nan
)
df['median_playtime_change'] = np.where(
    df['median_playtime_forever'] > 0,
    (df['median_playtime_2weeks'] - df['median_playtime_forever'])
    / df['median_playtime_forever'],
    np.nan
)

# df.drop(columns=['genres_'], inplace=True)
# df.drop(columns=['categories_'], inplace=True)

df.rename(columns={'developers': 'developer'}, inplace=True)
df.rename(columns={'publishers': 'publisher'}, inplace=True)

# 去除合併時產生的欄位
df.drop(columns=['genres_orig', 'categories_orig', 'genres_clean', 'categories_clean'], inplace=True)

# 從折扣反推遊戲原價
df['original_price'] = df['price'] / (1 - df['discount(%)'].replace(1, 0.99))

# 顯示欄位資訊
cols_to_show = [c for c in df.columns if not (c.startswith('genres_') or c.startswith('categories_'))]
df[cols_to_show].info()

In [ ]:
df['release_date_dt'] = pd.to_datetime(df['release_date'], errors='coerce')
df = df.dropna(subset=['release_date'])

df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
# 移除無法轉換的資料（包含僅有年份）
df = df.dropna(subset=['release_date_dt'])
# 只針對有「日」的資料取月份
mask = df['release_date'].astype(str).str.len() > 7
df.loc[mask, 'release_month'] = df.loc[mask, 'release_date_dt'].dt.month


# 確保 recommendations 是數值
df['recommendations'] = pd.to_numeric(df['recommendations'], errors='coerce').fillna(0)

# 新增標籤：是否有推薦數
df['recommendations_not_zero'] = (df['recommendations'] > 0).astype(int)

In [ ]:
df.head(5)

# 市場分析一：Steam市場整體

In [ ]:
# 1-1. 按照年度計算遊戲數量與年度成長率
year_stats = df.groupby('release_year').size().reset_index(name='game_count')
year_stats = year_stats.sort_values('release_year')
year_stats['growth_rate'] = year_stats['game_count'].pct_change()
year_stats['growth_rate_pct'] = year_stats['growth_rate'].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else "-")

year_stats[['release_year', 'game_count', 'growth_rate_pct']]

In [ ]:
# 1-2. 根據年度和遊戲種類(release_year, genres)分類，分析遊戲類型變化趨勢
genre_cols = [c for c in df.columns if c.startswith('genre_')]
genre_year = df.groupby('release_year')[genre_cols].sum()

genre_year['total_per_year'] = genre_year.sum(axis=1)
genre_year.loc['total_per_type'] = genre_year.sum(axis=0)

# 顯示完整欄位
pd.set_option('display.max_columns', None)
genre_year

In [ ]:
# 前 10 熱門類別
genre_cols = [c for c in df.columns if c.startswith('genre_')]
genre_year = df.groupby('release_year')[genre_cols].sum()

total_genre_counts = genre_year.sum(axis=0).sort_values(ascending=False)
top_genres = total_genre_counts.head(10).index

genre_year_top10 = genre_year[top_genres]
genre_year_top10

In [ ]:
# 1-3. 計算定價(price)和推薦數(recommendations)之間的相關係數
corr_price_to_recommendations = df[['price', 'recommendations']].corr()
corr_price_to_recommendations
# 價格和推薦數的相關性極低

In [ ]:
# 1-3-1. 檢查付費遊戲定價(price)和推薦數(recommendations)、開發商(developers)、發行商(publisher)之間的相關係數

# Filter Paid games (Free/paid == 1)
df_paid = df[df['Free/paid'] == 1].copy()

# 1. Price vs Recommendations
corr_price_recs = df_paid['price'].corr(df_paid['recommendations'])
print(f"Correlation (Price vs Recommendations): {corr_price_recs:.4f}")

# 2. Price vs Developer & Publisher
dev_mean_price = df_paid.groupby('developer')['price'].transform('mean')
pub_mean_price = df_paid.groupby('publisher')['price'].transform('mean')

corr_price_dev = df_paid['price'].corr(dev_mean_price)
corr_price_pub = df_paid['price'].corr(pub_mean_price)

print(f"Correlation (Price vs Developer Avg Price): {corr_price_dev:.4f}")
print(f"Correlation (Price vs Publisher Avg Price): {corr_price_pub:.4f}")
# 計算「相同開發商的平均價格」時，包含了該款遊戲本身。由於資料集中有高達 82.79% 的開發商僅發行過 1 款付費遊戲，故相關係數接近於1。

# 市場分析二：價格

In [ ]:
# 2-1 將定價分群(區間為10美元)，找出價格與推薦數從無關→較高相關性的價格區間
df_paid = df[(df['price'].notna()) & (df['price'] > 0)].copy()

bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 9999]
labels = [
    'Under $10', '$10 - $20', '$20 - $30', '$30 - $40',
    '$40 - $50', '$50 - $60', '$60 - $70', '$70 - $80',
    '$80 - $90', '$90 - $100', 'Over $100'
]
df_paid['price_bin'] = pd.cut(df_paid['price'], bins=bins, labels=labels, right=False)
bin_stats = df_paid.groupby('price_bin', observed=False).agg({
    'appid': 'count',
    'price': 'mean',
    'recommendations': 'mean'
}).rename(columns={'appid': 'count', 'price': 'avg_price', 'recommendations': 'avg_recs'})

corrs = []
for label in labels:
    subset = df_paid[df_paid['price_bin'] == label]
    if len(subset) > 10:
        r = subset['price'].corr(subset['recommendations'])
    else:
        r = np.nan
    corrs.append(r)

bin_stats['correlation_price_recs'] = corrs
print(bin_stats.to_markdown())

In [ ]:
# 2-1 低價位數量太多，增加分群點($5)
df_paid = df[(df['price'].notna()) & (df['price'] > 0)].copy()

bins = [0, 5, 10, 20, 30, 50, 70, 9999]
labels = [
    'Under $5', '$5 - $10', '$10 - $20', '$20 - $30',
    '$30 - $50', '$50 - $70', 'Over $70'
]
df_paid['price_bin'] = pd.cut(df_paid['price'], bins=bins, labels=labels, right=False)
bin_stats = df_paid.groupby('price_bin', observed=False).agg({
    'appid': 'count',
    'price': 'mean',
    'recommendations': 'mean'
}).rename(columns={'appid': 'count', 'price': 'avg_price', 'recommendations': 'avg_recs'})

corrs = []
for label in labels:
    subset = df_paid[df_paid['price_bin'] == label]
    if len(subset) > 10:
        r = subset['price'].corr(subset['recommendations'])
    else:
        r = np.nan
    corrs.append(r)

bin_stats['correlation_price_recs'] = corrs
print(bin_stats.to_markdown())

In [ ]:
# 2-1 低價位區增加分群點($5)，只針對有推薦的遊戲
df_active = df[df['recommendations'] > 0].copy()
df_active_paid = df_active[df_active['Free/paid'] == 1][(df['price'].notna())].copy()

bins = [0, 5, 10, 20, 30, 50, 70, 9999]
labels = [
    'Under $5', '$5 - $10', '$10 - $20', '$20 - $30',
    '$30 - $50', '$50 - $70', 'Over $70'
]
df_active_paid['price_bin'] = pd.cut(df_active_paid['price'], bins=bins, labels=labels, right=False)
bin_stats = df_active_paid.groupby('price_bin', observed=False).agg({
    'appid': 'count',
    'price': 'mean',
    'recommendations': 'mean'
}).rename(columns={'appid': 'count', 'price': 'avg_price', 'recommendations': 'avg_recs'})

corrs = []
for label in labels:
    subset = df_active_paid[df_active_paid['price_bin'] == label]
    if len(subset) > 10:
        r = subset['price'].corr(subset['recommendations'])
    else:
        r = np.nan
    corrs.append(r)

bin_stats['correlation_price_recs'] = corrs
print(bin_stats.to_markdown())

In [ ]:
# 關於遊戲發行時間點
# 簡單評估：最高與最低均值的月份
df_active = df[df['recommendations'] > 0].copy()
print(f"有效樣本數 (Recommendations > 0): {len(df_active)} / {len(df)}")

monthly_stats = df_active.groupby('release_month')['recommendations'].agg(['count', 'mean', 'median', 'std']).reset_index()
monthly_stats = monthly_stats.round(0)

print("月份整體統計:")
print(monthly_stats.to_string(index=False))

max_month = monthly_stats.loc[monthly_stats['mean'].idxmax()]
min_month = monthly_stats.loc[monthly_stats['mean'].idxmin()]
print(f"\n平均推薦數最高的月份: {int(max_month['release_month'])}月 (Avg: {max_month['mean']})")
print(f"平均推薦數最低的月份: {int(min_month['release_month'])}月 (Avg: {min_month['mean']})")

In [ ]:
df_active['price_bin'] = pd.cut(df_active['price'], bins=bins, labels=labels, right=False)
bin_stats = df_active.groupby('price_bin', observed=False).agg({
    'appid': 'count',
    'price': 'mean',
    'recommendations': 'mean'
}).rename(columns={'appid': 'count', 'price': 'avg_price', 'recommendations': 'avg_recs'})

corrs = []
for label in labels:
    subset = df_active[df_active['price_bin'] == label]
    if len(subset) > 10:
        r = subset['price'].corr(subset['recommendations'])
    else:
        r = np.nan
    corrs.append(r)

bin_stats['correlation_price_recs'] = corrs
print(bin_stats.to_markdown())

In [ ]:
# 2-2. 根據發行月份檢查與推薦數的關聯 (僅限推薦數 > 0)
# 過濾：只留下有推薦數的遊戲
df_active = df[df['recommendations'] > 0].copy()
print(f"有效樣本數 (Recommendations > 0): {len(df_active)} / {len(df)}")

def monthly_stats(df):
    monthly_stats = df.groupby('release_month')['recommendations'].agg(['count', 'mean', 'median', 'std']).reset_index()
    monthly_stats = monthly_stats.round(2)

    max_month = monthly_stats.loc[monthly_stats['mean'].idxmax()]
    min_month = monthly_stats.loc[monthly_stats['mean'].idxmin()]
    return monthly_stats

    print(f"\n平均推薦數最高的月份: {int(max_month['release_month'])}月 (Avg: {max_month['mean']})")
    print(f"平均推薦數最低的月份: {int(min_month['release_month'])}月 (Avg: {min_month['mean']})")
monthly_stats(df_active)

In [ ]:
# 2-2-1. 分成免費與付費遊戲，檢查發行月份與推薦數的關聯 (僅限推薦數 > 0)
df_active_free = df_active[df_active['Free/paid'] == 0].copy()
df_active_paid = df_active[df_active['Free/paid'] == 1].copy()
print(f"免費樣本數 (['Free/paid'] == 0): {len(df_active_free)} / {len(df)}")
print(f"免費樣本數 (['Free/paid'] == 1): {len(df_active_paid)} / {len(df)}")

print("[免費遊戲 (Free)] 月份統計:")
monthly_stats(df_active_free)

print("[付費遊戲 (Paid)] 月份統計:")
monthly_stats(df_active_paid)

In [ ]:
# 2-2-3 價格分群法 (僅限推薦數 > 0)
# 定義分箱區間與標籤
bins = [0, 5, 10, 20, 30, 50, 70, 9999]
labels = ['very_low', 'low', 'mid_low', 'mid', 'high', 'very_high', 'out_of_market']

# 建立價格分組欄位
df_active['price_bin'] = pd.cut(df_active['price'], bins=bins, labels=labels, right=False)

price_groups = {}  # 建立空字典
for label in labels:
    price_groups[label] = df_active[df_active['price_bin'] == label].copy()

for i in range(len(labels)):
    print(labels[i],":",len(price_groups[labels[i]]))
    print(monthly_stats(price_groups[labels[i]]))
    print('--'*30)

# 市場分析三：獨立遊戲 Indie Game

In [ ]:
df_indie = df[df['genres_Indie'] == 1].copy()

In [ ]:
def year_stats(df):
    stats = df.groupby('release_year').agg(
        count=('price', 'size'),
        avg_price=('price', 'mean'),
        median_price=('price', 'median')
    ).reset_index()
    stats['growth_rate'] = stats['count'].pct_change() * 100
    return stats

year_stats(df_indie)

In [ ]:
# 3-1. 獨立遊戲(Indie Game)按照年度順序計算發行數量與年度成長率 ---
indie_yearly = df_indie.groupby('release_year').size().reset_index(name='count')
indie_yearly['growth_rate'] = indie_yearly['count'].pct_change() * 100

indie_yearly

In [ ]:
# 3-2. 分析付費(paid)的Indie Game發行數、年度成長率、價格變化
df_indie_paid = df_indie[df_active_paid_mask := (df_indie['price'] > 0)].copy()
indie_paid_yearly = df_indie_paid.groupby('release_year').agg(
    count=('price', 'size'),
    avg_price=('price', 'mean'),
    median_price=('price', 'median')
).reset_index()
indie_paid_yearly['growth_rate'] = indie_paid_yearly['count'].pct_change() * 100
indie_paid_yearly.style.format({"growth_rate": "{:.2%}"})

indie_paid_yearly

In [ ]:
# 3-3. 分析推薦數>0的Indie Game
df_indie_active = df_indie[df_indie['recommendations'] > 0].copy()
indie_active_yearly = df_indie_active.groupby('release_year').size().reset_index(name='count')
indie_active_yearly['growth_rate'] = indie_active_yearly['count'].pct_change() * 100
indie_active_yearly

# 2025年剛過完，有可能是2025年發行遊戲的推薦時間還不夠，檢查看看
# 2025年成長下降的因素：受「時間累積不足」影響
# Focus on 2023, 2024, 2025
years = [2023, 2024, 2025]
df_recent = df_indie[df_indie['release_year'].isin(years)].copy()

# Calculate Active Rate (Rec > 0) per month per year
monthly_analysis = df_recent.groupby(['release_year', 'release_month']).agg(
    total_count=('recommendations', 'size'),
    active_count=('recommendations', lambda x: (x > 0).sum())
).reset_index()

monthly_analysis['active_rate'] = (monthly_analysis['active_count'] / monthly_analysis['total_count']) * 100

print("\n[各月份有效推薦比例 (%) 比較]")
print("-" * 100)
pivot_table = monthly_analysis.pivot(index='release_month', columns='release_year', values='active_rate')
print(pivot_table.round(2))

print("\n" + "=" * 100)
print("分析結論")
print("-" * 100)

# Analyze decay in 2025
h_2024 = monthly_analysis[(monthly_analysis['release_year'] == 2024)]['active_rate'].mean()
h_2025 = monthly_analysis[(monthly_analysis['release_year'] == 2025)]['active_rate'].mean()

print(f"2024 推薦平均有效率: {h_2024:.2f}%")
print(f"2025 推薦平均有效率: {h_2025:.2f}%")
print("2025下半年推薦數有效率明顯下降")
# 如果在半年或一年後再次確認數據，2025 年的「有效率」可能會回升到與 2024 年相近的水平（約 11-12%）。

In [ ]:
# 3-4. 列出Indie Game推薦數前10名遊戲
print("\n[3-4] 獨立遊戲推薦數排行榜 Top 10:")
top_10_indie = df_indie.sort_values('recommendations', ascending=False).head(10)
top_10_indie[['name','release_year','price','recommendations']]


In [ ]:
# 3-5. 分析Indie Game推薦數變化，確認高熱門度(高推薦數)的標準
print("\n" + "=" * 80)
print("[3-5] 獨立遊戲推薦數分布分析 (尋找高熱門度標準)")
print("-" * 80)

rec_stats = df_indie['recommendations'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.97, 0.99, 0.999])
print(rec_stats)

# 根據分析結果定義「高熱門度」門檻
threshold = 1100
df_indie_high_pop = df_indie[df_indie['recommendations'] >= threshold].copy()

print(f"\n定義「高熱門度」門檻為: {threshold} 推薦數")
print(f"符合門檻的遊戲數量: {len(df_indie_high_pop)} 款 (佔獨立遊戲總數的 {len(df_indie_high_pop)/len(df_indie)*100:.2f}%)")


In [ ]:
# 3-5-1. 針對高熱門度Indie Game分析年度數據變化
# 3-5-2. 針對高熱門度Indie Game分析年度發行數、年度成長率、價格變化
high_pop_yearly = df_indie_high_pop.groupby('release_year').agg(
    count=('recommendations', 'size'),
    avg_rec=('recommendations', 'mean'),
    median_rec=('recommendations', 'median'),
    avg_price=('price', 'mean'),
    median_price=('price', 'median')
).reset_index()

high_pop_yearly['growth_rate'] = high_pop_yearly['count'].pct_change() * 100
high_pop_yearly

In [ ]:
# 發行 3 款以上遊戲的開發商相關係數分析
dev_stats = df.groupby('developer').agg({
    'appid': 'count',
    'price': 'mean',
    'recommendations': 'mean'
}).rename(columns={
    'appid': 'game_count',
    'price': 'avg_price',
    'recommendations': 'avg_recommendations'
})

active_devs = dev_stats[dev_stats['game_count'] > 3]
dev_corr = active_devs.corr()
print("\nCorrelation Matrix (Active Developers):")
print(dev_corr.to_markdown())
# 遊戲發行量與價格、推薦數基本上無關
# 價格和推薦數有低度正相關，定價較高的開發商，其遊戲獲得的平均推薦數略高。

In [ ]:
# 算跟開發商的相關係數
dev_stats = df.groupby('developer').agg({
    'appid': 'count',
    'price': 'mean',
    'recommendations': 'mean'
}).rename(columns={'appid': 'game_count', 'price': 'avg_price', 'recommendations': 'avg_recs'})

active_devs = dev_stats[dev_stats['game_count'] > 3].copy()

# 2. Binning Developers by their Average Price
active_devs['price_bin'] = pd.cut(active_devs['avg_price'], bins=bins, labels=labels, right=False)

dev_bin_stats = active_devs.groupby('price_bin', observed=False).agg({
    'game_count': 'count',
    'avg_price': 'mean',
    'avg_recs': 'mean'
}).rename(columns={'game_count': 'dev_count'})

dev_corrs = []
for label in labels:
    subset = active_devs[active_devs['price_bin'] == label]
    if len(subset) > 5: # Slightly lower threshold for devs as count is smaller
        r = subset['avg_price'].corr(subset['avg_recs'])
    else:
        r = np.nan
    dev_corrs.append(r)

dev_bin_stats['corr_avg_price_avg_recs'] = dev_corrs
print(dev_bin_stats.to_markdown())

In [ ]:
# 3-5-3. 列出高熱門度Indie Game的開發商和發行商，檢查是否有明顯優秀的廠商
print("\n" + "=" * 80)
print("[3-5-3] 高熱門度獨立遊戲之優勢廠商分析 (Top 10)")
print("-" * 80)

def top_entities(df, entity_col):
    entity_stats = df.groupby(entity_col).agg(
        game_count=('recommendations', 'size'),
        total_rec=('recommendations', 'sum'),
        avg_rec=('recommendations', 'mean')
    ).sort_values('game_count', ascending=False).head(10)
    return entity_stats

print("\n[最成功的開發商 (依高品質遊戲量排序)]")
print(top_entities(df_indie_high_pop, 'developer'))

print("\n[最成功的發行商 (依高品質遊戲量排序)]")
print(top_entities(df_indie_high_pop, 'publisher'))


In [ ]:
def games_by_entities(df, entity_list, entity_col):
    # 將每個名稱轉成正則 pattern，忽略大小寫
    pattern = '|'.join(entity_list)
    # 建立布林遮罩
    mask = df[entity_col].str.contains(pattern, case=False, na=False)
    # 篩選並依 recommendations 排序
    df_filtered = df[mask].sort_values('developer').sort_values('recommendations', ascending=False)
    return df_filtered

# 開發商總推薦數大於50,000的
Landfall = games_by_entities(
    df,
    ['Landfall'],
    entity_col='developer'
)
Steel_Wool_Studios = games_by_entities(
    df,
    ['Steel Wool Studios'],
    entity_col='developer'
)


# 顯示結果
print('-'*120)
print(Landfall[['name', 'release_date', 'recommendations', 'price', 'developer', 'publisher']])
print('-'*120)
print(Steel_Wool_Studios[['name', 'release_date', 'recommendations', 'price', 'developer', 'publisher']])
print('-'*120)

In [ ]:
# 發行商總推薦數最高的
# Devolver Digital是美國共同發行商，專注於獨立遊戲的發行工作。
Devolver_Digital = games_by_entities(
    df,
    ['Devolver Digital'],
    entity_col='publisher'
)

# 顯示結果
print('-'*120)
print(Devolver_Digital[['name', 'release_date', 'recommendations', 'price', 'developer', 'publisher']])
print('-'*120)

In [ ]:
top_10_indie = df_indie.sort_values('recommendations', ascending=False).head(10).copy()

# 幫資料表加上中文名
chinese_name_map = {}
chinese_name_map.update({
    'Valheim':'瓦爾海姆',
    'Lethal Company': '致命公司',
    'Palworld': '幻獸帕魯',
    'Hollow Knight: Silksong':'空洞騎士：絲之歌',
    'Raft':'Raft',
    '7 Days to Die':'七日殺',
    'Sons Of The Forest': '森林之子',
    'Vampire Survivors': '吸血鬼倖存者',
    'Schedule I': 'Schedule I (販毒模擬器)',
    'Ready or Not':'嚴陣以待'
})

top_10_indie['chinese_name'] = top_10_indie['name'].map(chinese_name_map)
top_10_indie['release_date_dt'] = pd.to_datetime(top_10_indie['release_date'], errors='coerce')

# Create the formatted table content
header = f"{'排名':<5} {'推薦數':<10} {'年度':<6} {'價格':<8} {'中文名稱':<25} {'英文名稱'}\n"
separator = "-" * 100 + "\n"
rows = []
for i, (_, row) in enumerate(top_10_indie.iterrows(), 1):
    rows.append(f"{i:<5} {int(row['recommendations']):<10,} {row['release_year']:<6} ${row['price']:<7.2f} {row['chinese_name']:<25} {row['name']}")

file_content = "🎮 獨立遊戲推薦數排行榜 Top 10 (2021-2025)\n" + separator + header + separator + "\n".join(rows)

# Save to text file
with open("Top10_indie_games.txt", 'w', encoding='utf-8') as f:
    f.write(file_content)

top_10_indie[['name','chinese_name','release_year','price','recommendations']]

# 市場分析四：免費遊戲

In [ ]:
# 4-1. 列出推薦數>0的免費遊戲，按照年度順序計算遊戲數量與年度成長率
df_active_free = df_active[df_active['Free/paid'] == 0].copy()
df_active_free
free_yearly = df_active_free.groupby('release_year').size().reset_index(name='count')
free_yearly['growth_rate'] = free_yearly['count'].pct_change() * 100

print(f"推薦數>0的免費遊戲數量: {len(df_active_free)} 款")

In [ ]:
# 看免費遊戲的推薦數分布
def get_dist(series):
    return series.describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.97, 0.99, 0.995, 0.999])

get_dist(df_active_free['recommendations'])

In [ ]:
# 4-2. 照 2-2 的作法，針對推薦數>0的免費遊戲，分析發行月份和推薦數的關聯
monthly_stats(df_active_free)
# 樣本數太少沒有參考價值

In [ ]:
df_free_hpop = df_active_free[df_active_free['recommendations'] > 1100].copy()
df_free_hpop = df_free_hpop.sort_values('recommendations', ascending=False)

cols_to_show = ['name', 'release_year', 'release_date', 'recommendations', 'genres', 'categories']
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print(f"\n定義「高熱門度」門檻為: {threshold} 推薦數")
print(f"推薦數>1100的免費遊戲數量: {len(df_free_hpop)} 款")
df_free_hpop[cols_to_show].head(10)

In [ ]:
# 4-4. 營運模式深度查證：
# 運用LLM協助搜尋確認後，以標籤分類商業模式匯出
def categorize_business_model(row):
    name = str(row['name']).lower()

    categories = str(row.get('categories', '')).lower()
    genres = str(row.get('genres', '')).lower()

    # --- A. 修正：這些在資料集中標記為 0 但實則為付費大作 ---
    premium_fix = [
        'football manager', 'thunder tier one', 'atelier ryza 2',
        'little nightmares', 'atelier ryza 3', 'battlefield 2042',
        'Nobody'
    ]

    models = []

    if any(k in name for k in premium_fix) or 'premium_keywords' in name:
        models.append('資料集標記異常/可能為限時免費')

    # --- B. 大型 IP 入口與版本更新 (IP Portal/Version Update) ---
    # 邏輯：遊戲主體免費下載（啟動器），但內容需訂閱或購買 DLC（如 COD, Halo）。
    ip_keywords = [
        'command & conquer', 'total war', 'warhammer', 'ea sports',
        'efootball', 'samurai shodown', 'battlefield', 'call of duty',
        'final fantasy', 'game of thrones', 'halo'
    ]
    if any(k in name for k in ip_keywords):
        models.append('大型 IP 入口/版本更新 (IP Portal)')

    # --- C. 章節/試玩吸引術 (Episodic/Prologue/Pass) ---
    # 邏輯：免費提供第一章或好友通行證，誘導購買完整版。
    prologue_keywords = [
        'prologue', 'episode', 'demo', 'nexus', 'chapter',
        "friend's pass", 'trial', 'poppy'
    ]
    if any(k in name for k in prologue_keywords):
        models.append('章節/試玩 (Episodic/Prologue)')

    # --- D. 線上營運與內購制 (Service-based F2P / IAP) ---
    # 邏輯：有 [In-App Purchases] 標籤，或已知 F2P 轉型作品。
    # 修正：補強 Hired Ops, XERA, VAIL, BROKE PROTOCOL 等內購制競技遊戲。
    f2p_iap_names = [
        'naraka', 'fragpunk', 'stormgate', 'fear surrounds',
        'zenith', 'hired ops', 'xera', 'vail', 'broke protocol'
    ]
    if ('in-app purchases' in categories or
        'massively multiplayer' in genres or
        any(k in name for k in f2p_iap_names)):
        models.append('線上營運/內購制 (Service-based F2P)')

    # --- 特別檢查：真正免費的線上遊戲 ---
    f2p_verified = ['world war 3', 'quake champions', 'lost ark', 'stumble guys']
    if any(k in name for k in f2p_verified):
        models.append('線上營運/內購制 (Service-based F2P)')

    special = ['knightfall']
    if any(k in name for k in special):
        models.append('2024年開放永久免費')

    if not models:
        return '真正免費 (Experiment/Legacy)'

    return ' | '.join(models)

chinese_name_map.update({
    'Call of Duty®': '決勝時刻',
    'NARAKA: BLADEPOINT': '永劫無間',
    "It Takes Two Friend's Pass": '雙人成行 好友通行證',
    'FIFA 22': 'FIFA 22',
    'EA SPORTS™ FIFA 23': 'EA SPORTS™ FIFA 23',
    'EA SPORTS FC™ 24': 'EA SPORTS FC™ 24',
    'Lost Ark': '失落的方舟',
    'Poppy Playtime': '波比的遊戲時間',
    'Poppy Playtime - Chapter 1': '波比的遊戲時間：第一章',
    'Halo Infinite': '最後一戰：無限',
    'Stumble Guys': 'Stumble Guys',
    'Call of Duty®: Warzone™': '決勝時刻：現代戰域',
    'Battlefield™ 2042': '戰地風雲 2042',
    'Battlefield™ REDSEC': '戰地風雲 REDSEC (資料集標記)',
    'Nobody - The Turnaround': '大多數',
    'Madden NFL 24': '麥登橄欖球 24',
    'Total War: PHARAOH DYNASTIES': '全軍破敵：法老 王朝',
    'Yi Xian: The Cultivation Card Game': '弈仙牌',
    'Town of Salem 2': '薩勒姆小鎮 2',
    'Zenith: Nexus': '齊尼斯：結點',
    'Knightfall: A Daring Journey': '騎士隕落：英勇旅程',
    'Stormgate': '風暴之門',
    'FragPunk': '狂整 (FragPunk)',
    'SAMURAI SHODOWN': '侍魂 曉',
    '江湖客栈-The Jianghu': '江湖客栈',
    'Vigor': '維戈 (Vigor)',
    'Game of Thrones: Kingsroad': '權力的遊戲：國王之路',
    'eFootball™': 'eFootball™ (實況足球)',
    'Fear Surrounds': '恐懼之間',
    'Hired Ops': '傭兵行動',
    'XERA: Survival': 'XERA: 生存',
    'VAIL: Extraction': 'VAIL: 撤離',
    'BROKE PROTOCOL': '破碎協議',
    'Atelier Ryza 3: Alchemist of the End & the Secret Key': '萊莎的鍊金工房３ ～終結之鍊金術士與秘密鑰匙～',
    'Sheepy: A Short Adventure': '小羊：一場短篇冒險',
    # 終極動員令系列
    'Command & Conquer™ Generals': '終極動員令：將軍',
    'Command & Conquer™ Generals Zero Hour': '終極動員令：將軍 絕命時刻',
    'Command & Conquer Red Alert™ 2 and Yuri’s Revenge™': '終極動員令：紅色警戒 2 與 尤里的復仇',
    'Command & Conquer™ Tiberian Sun™ and Firestorm™': '終極動員令：泰伯倫之日 與 火線爭鋒',
    'Command & Conquer™ Red Alert™ 3': '終極動員令：紅色警戒 3',
    'Command & Conquer 3: Tiberium Wars': '終極動員令 3：泰伯倫戰爭',
    'World War 3': '第三次世界大戰',
    'Football Manager 2024': '足球經理 2024',
    'Football Manager 2023': '足球經理 2023',
    'Football Manager 2022': '足球經理 2022',
    'Quake Champions': '雷神之鎚：冠軍',
    'Thunder Tier One': '雷霆一號',
    'Atelier Ryza 2: Lost Legends & the Secret Fairy': '萊莎的鍊金工房 2 ～失落傳說與秘密妖精～',
    'Little Nightmares Enhanced Edition': '小小夢魘 強化版'
})

# 應用分類與中文名稱
df_free_hpop['chinese_name'] = df_free_hpop['name'].map(chinese_name_map).fillna(df_free_hpop['name'])
df_free_hpop['business_model'] = df_free_hpop.apply(categorize_business_model, axis=1)

# 整理輸出欄位 (加入中文名稱)
final_cols = ['name', 'chinese_name', 'recommendations', 'release_year', 'business_model']
df_final_output = df_free_hpop[final_cols]

df_final_output.to_csv('free_games_hotpop.csv', encoding='utf-8')
df_final_output.head(5)

In [ ]:
# 整理遊戲中文名 (包含異常值中的典型代表)
chinese_name_map.update({
    'Black Myth: Wukong': '黑神話：悟空', 'ELDEN RING': '艾爾登法環', 'Lethal Company': '致命公司',
    'Palworld': '幻獸帕魯', 'R.E.P.O.': 'R.E.P.O. (無官方中文名)', 'Battlefield™ 2042': '戰地風雲 2042',
    'Sons Of The Forest': '森林之子', 'Vampire Survivors': '吸血鬼倖存者', 'Schedule I': 'Schedule I (販毒模擬器)',
    'Forza Horizon 5': '極限競速 地平線 5', 'Battlefield™ 6': '戰地風雲 6', 'PEAK': 'PEAK (巔峰)',
    'It Takes Two': '雙人成行', '鬼谷八荒 Tale of Immortal': '鬼谷八荒', 'Monster Hunter Wilds': '魔物獵人 荒野',
    'ARC Raiders': 'ARC Raiders', 'Stray': 'Stray (浪貓)', 'Path of Exile 2': '流放之路 2',
    'Clair Obscur: Expedition 33': '光與影：33號遠征隊', 'Balatro': '小丑牌',
    'Ready or Not': '嚴陣以待', 'Content Warning': '內容警告', 'Party Animals': '動物派對',
    'Enshrouded': '霧鎖王國', 'Pacific Drive': '太平洋駕駛', 'Buckshot Roulette': '惡魔輪盤',
    'Manor Lords': '莊園領主', 'Bodycam': '攝像機 (Bodycam)', 'Bodycam ': '攝像機 (Bodycam)',
    'V Rising': '夜族崛起', 'Last Epoch': '最後紀元', 'Content Warning': '內容警告',

    # 運動與模擬類 (Football Manager & Madden)
    'Football Manager 2024': '足球經理 2024',
    'Football Manager 2023': '足球經理 2023',
    'Football Manager 2022': '足球經理 2022',
    'Madden NFL 24': '麥登橄欖球 24',
    'Forza Horizon 5': '極限競速 地平線 5',

    # 動作、射擊與生存類 (Action & FPS)
    'World War 3': '第三次世界大戰',
    'Quake Champions': '雷神之鎚：冠軍',
    'Thunder Tier One': '雷霆一號',
    'Hired Ops': '傭兵行動',
    'VAIL: Extraction': 'VAIL: 撤離',
    'XERA: Survival': 'XERA: 生存',
    'BROKE PROTOCOL': '破碎協議',
    'FragPunk': '界外狂潮 (FragPunk)',
    '7554': '7554 (越南抗法英雄傳)',

    # 冒險與 RPG 類 (RPG & Adventure)
    'Atelier Ryza 2: Lost Legends & the Secret Fairy': '萊莎的鍊金工房 2 ～失落傳說與秘密妖精～',
    'Atelier Ryza 3: Alchemist of the End & the Secret Key': '萊莎的鍊金工房 3 ～終結之鍊金術士與秘密鑰匙～',
    'Little Nightmares Enhanced Edition': '小小夢魘 強化版',
    'Poppy Playtime - Chapter 1': '波比的遊戲時間：第一章',
    'Zenith: Nexus': '齊尼斯：結點',
    'Stormgate': '風暴之門',

    # 策略與經典 IP (Strategy & IP Portal)
    'Total War: PHARAOH DYNASTIES': '全軍破敵：法老 王朝',
    'Command & Conquer™ Red Alert™ 3': '終極動員令：紅色警戒 3',
    'Command & Conquer 3: Tiberium Wars': '終極動員令 3：泰伯倫戰爭',
    'Command & Conquer™ Generals': '終極動員令：將軍',
    'Command & Conquer™ Generals Zero Hour': '終極動員令：將軍 絕命時刻',

    # 其他在清單中的項目
    'Fear Surrounds': '恐懼之間',
    'Yi Xian: The Cultivation Card Game': '弈仙牌',
    'Town of Salem 2': '薩勒姆小鎮 2',
    'Knightfall: A Daring Journey': '騎士隕落：英勇旅程',
    'Sheepy: A Short Adventure': '小羊：一場短篇冒險',

    # 核心遺漏項目
    'HELLDIVERS™ 2': '絕地戰兵 2',
    'Forza Horizon 5': '極限競速 地平線 5',
    '7554': '7554 (越南抗法英雄傳)',
    'Battlefield™ 6': '戰地風雲 6 (資料集標記)', # 實際上應為 2042，但依據資料集名稱標記

    # 試玩與章節類
    'Life is Strange - Episode 1': '奇妙人生：第一集',
    'Resident Evil Village Gameplay Demo': '惡靈古堡 8：村莊 遊戲試玩版',
    'The Matrix Awakens: An Unreal Engine 5 Experience': '駭客任務 覺醒：虛幻引擎 5 體驗',

    # 射擊與線上營運類 (F2P/Premium)
    'Apex Legends': ' Apex 英雄',
    'Destiny 2': '天命 2',
    'PUBG: BATTLEGROUNDS': '絕地求生',
    'Overwatch® 2': '鬥陣特攻 2',
    'Counter-Strike 2': '絕對武力 2',

    # 其他在 outliers.csv 中出現的項目
    'Sea of Thieves': '盜賊之海',
    'Dead by Daylight': '黎明死線',
    'Rust': ' Rust (腐蝕)',
    'The Forest': '森林',
    "Baldur's Gate 3": '柏德之門3',
    'Hogwarts Legacy': '霍格華茲的傳承',
    'HELLDIVERS™ 2': '絕地戰兵 2',
    'Cyberpunk 2077': '賽博朋克 2077',
    'ELDEN RING': '艾爾登法環',
    'Starfield': '星空',
    'Forza Horizon 5': '極限競速 地平線 5',
    'Ready or Not': '準備好了沒 (Ready or Not)',
    'Rust': '腐蝕 (Rust)',

    # 資料集內提到的其他項目
    '7554': '7554 (越南抗法英雄傳)',
    'Battlefield™ 6': '戰地風雲 6 (資料集標記名稱)'
})

# 市場分析五：代表性作品

In [ ]:
df['recommendations'] = pd.to_numeric(df['recommendations'], errors='coerce').fillna(0)
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')

# Filter for Paid Games (Price > 0)
df_paid = df[df['price'] > 0].copy()
paid_indie = df_paid[df_paid['genres_Indie'] == 1].copy()
paid_non_indie = df_paid[df_paid['genres_Indie'] == 0].copy()

In [ ]:
# 5-1 付費遊戲高熱門度分析 (標準訂定)
def get_dist(series):
    return series.describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.97, 0.99, 0.995, 0.999])

dist_all = get_dist(df_paid['recommendations'])
dist_indie = get_dist(paid_indie['recommendations'])
dist_non_indie = get_dist(paid_non_indie['recommendations'])

print(f"{'百分位':<10} {'所有付費':<15} {'獨立付費':<15} {'非獨立付費 (AA/3A)'}")
print("-" * 60)
for p in ['50%', '90%', '95%', '97%', '99%', '99.5%', '99.9%']:
    print(f"{p:<10} {dist_all[p]:<15.0f} {dist_indie[p]:<15.0f} {dist_non_indie[p]:<15.0f}")

# 定義門檻 (PR97 左右)
high_pop_threshold = 2000
df_high_pop = df_paid[df_paid['recommendations'] >= high_pop_threshold].copy()

print(f"\n✅ 最終『付費遊戲高熱門度』標準為: {high_pop_threshold} 推薦數")
print(f"符合標竿的遊戲數量: {len(df_high_pop)} 款 (佔總體付費遊戲之 {len(df_high_pop)/len(df_paid)*100:.2f}%)")


In [ ]:
# 5-2. 列出推薦數異常值
# 使用 PR99.5 作為「現象級作品」門檻
outlier_threshold = int(df_paid['recommendations'].quantile(0.995))
outlier_threshold = 20000

df_outliers = df_paid[df_paid['recommendations'] >= outlier_threshold].copy()

print(f"✅ 定義『現象級異常值 (Outlier)』標準為: {outlier_threshold} 推薦數 (PR99.5)")
print(f"全體付費遊戲中僅有 {len(df_outliers)} 款作品達到此巔峰標竿 (約佔 {len(df_outliers)/len(df_paid)*100:.2f}%)")

In [ ]:
# 列出這些「異常爆紅」的作品分類
df_outliers['categories'] = df_outliers['genres_Indie'].apply(lambda x: 'B. Phenomenal Indie/AA' if x == 1 else 'A. Major AAA')
df_outliers['chinese_name'] = df_outliers['name'].map(chinese_name_map)

# 顯示前 20 名異常值作為範例
top_outliers = df_outliers.sort_values('recommendations', ascending=False).head(20)

for cat in ['A. Major AAA', 'B. Phenomenal Indie/AA']:
    sub_df = top_outliers[top_outliers['categories'] == cat]
    if len(sub_df) > 0:
        print(f"\n[{cat}]")
        print("-" * 120)
        print(f"{'推薦數':<12} {'年度':<6} {'價格':<8} {'中文名稱':<25} {'英文名稱'}")
        for _, row in sub_df.iterrows():
            print(f"{int(row['recommendations']):<12,} {int(row['release_year']):<6} ${row['price']:<7.2f} {str(row['chinese_name']):<25} {row['name']}")

top_outliers.to_csv('outliers.csv',encoding='utf-8')

In [ ]:
# 5-3 分析高熱門作品非獨立遊戲的發行商、開發商集中度。
df_high_pop_non_indie = df_high_pop[df_high_pop['genres_Indie'] == 0].copy()

# A. 發行商分析
print("\n[Top 10 大型發行商 - 按遊戲量排序]")
pub_stats = df_high_pop_non_indie.groupby('publisher').agg(
    game_count=('name', 'count'),
    total_rec=('recommendations', 'sum'),
    avg_rec=('recommendations', 'mean')
).sort_values('game_count', ascending=False).head(10)
print(pub_stats)

# B. 開發商分析
print("\n[Top 10 核心開發商 - 按遊戲量排序]")
dev_stats = df_high_pop_non_indie.groupby('developer').agg(
    game_count=('name', 'count'),
    total_rec=('recommendations', 'sum'),
    avg_rec=('recommendations', 'mean')
).sort_values('game_count', ascending=False).head(10)
print(dev_stats)

In [ ]:
# 5-3-1. 卓越廠商定價策略 ---
for name, items in [("發行商", pub_stats.index), ("開發商", dev_stats.index)]:
    print(f"\n[{name} 定價中位數與範圍]")
    for item in items:
        col = 'publisher' if name == "發行商" else 'developer'
        prices = df_paid[df_paid[col] == item]['price']
        if not prices.empty:
            print(f"{item[:25]:<26} 中位數: ${prices.median():<6.2f} 範圍: ${prices.min():>5.2f} - ${prices.max():<6.2f}")

In [ ]:
# 5-4 得獎作品群集 (包含 TGA & IGA 2021-2025)
# 判斷得獎作品是否在Steam資料集內
mask = df_awards['name'].str.lower().isin(df['name'].str.lower())
df_existing = df_awards[mask]
df_missing = df_awards[~mask]

print("存在於 df 的遊戲數量:", mask.sum())
print("不存在於 df 的遊戲數量:", (~mask).sum())
df_awards[~mask]

year_mapping = df.drop_duplicates('appid').set_index('appid')['release_year']

df_awards['release_year'] = df_awards['appid'].map(year_mapping)

print(f"成功對應年份的數量: {df_awards['release_year'].notnull().sum()}")
print(df_awards[['appid', 'name', 'release_year']].head())

In [ ]:
df_awards['name_lower'] = df_awards['name'].str.lower()
df['name_lower'] = df['name'].str.lower()

# 只保留需要欄位
df_to_merge = df[['name_lower', 'appid']]
df_awards = df_awards.merge(
    df_to_merge,
    on='name_lower',
    how='left',
    suffixes=('', '_from_df')
)

# 將新的 appid 填入原欄位
df_awards['appid'] = df_awards['appid_from_df'].combine_first(df_awards['appid'])
df_awards = df_awards.drop(columns=['name_lower', 'appid_from_df'])


In [ ]:
# 得獎作品中在steam資料集內的數量
mask = df_awards['appid'].isin(df['appid'])
df_existing_awards = df_awards[mask]
df_missing_awards = df_awards[~mask]
print("存在於 df 的數量:", mask.sum())
print("不存在於 df 的數量:", (~mask).sum())

# steam資料集中在得獎名單內的數量
mask = df['appid'].isin(df_awards['appid'])
df_existing_awards = df[mask]
print("存在於 df 的數量:", mask.sum())

# 兩者數量不同，因為得獎名單可能有重複得獎的狀況，去重後實際遊戲列表為第二個

In [ ]:
monthly_stats(df_existing_awards)
# 數量太少，月份分析沒有統計意義

In [ ]:
# 5-5 折扣引流效果
# 本資料集沒有時間序列，只能知道取資料該時間點的折扣比例
mask = df['discount'] > 0
df_discount = df.loc[mask].copy()
df_no_discount = df.loc[~mask].copy()
df_discount['appid'].nunique()

In [ ]:
cols = [
    'discount(%)',
    'average_playtime_change',
    'median_playtime_change']

df_discount_corr = (df_discount[cols].dropna().corr(method='pearson'))
df_discount_corr

In [ ]:
df_discount_pa = df.loc[df['discount(%)'] > 0].copy()
print(df_discount)
df_discount_pa['discount_bin'] = pd.cut(
    df_discount_pa['discount(%)'],
    bins=[0, .3, .6, .9, 1.0],
    labels=['0–30%', '30–60%', '60–90%', '90–100%'],
    include_lowest=True

df_discount_pa.groupby('discount_bin', observed=True)[
    ['median_playtime_change', 'average_playtime_change']
].median()
# 太多-1無法看出規律(意思是絕大部分遊戲兩周內都沒有遊玩數無法計算)

In [ ]:
# 找出知名發行商與開發商的折扣策略

df_active_paid = df[(df['price'] > 0) & (df['recommendations'] > 0)].copy()

def check_top_10_results(df, entity_col):
    entity_stats = df.groupby(entity_col).agg(
        game_count=('appid', 'nunique'),
        total_rec=('recommendations', 'sum'),
        avg_rec=('recommendations', 'mean'),
        avg_price=('price', 'mean'),
        avg_discount=('discount(%)', 'mean')
    )

    entity_stats = entity_stats[entity_stats['game_count'] >= 3
                                ].sort_values('total_rec', ascending=False).head(10)

    top_games_list = []
    for entity in entity_stats.index:
        # 找出該廠商的所有遊戲，按推薦數排序取前三
        top_3 = df[df[entity_col] == entity].nlargest(3, 'recommendations')['name'].tolist()
        top_3_cn = [chinese_name_map.get(name, name) for name in top_3]
        top_games_list.append(" | ".join(top_3_cn))

    entity_stats['知名遊戲'] = top_games_list
    return entity_stats

pub_check = check_top_10_results(df_active_paid, 'publisher')
dev_check = check_top_10_results(df_active_paid, 'developer')


print("-" * 30)
print("🏆 前十大發行商策略驗證")
print("-" * 30)
display(pub_check.style.format({
    'avg_price': '${:.2f}',
    'avg_discount': '{:.1%}',
    'avg_rec': '{:,.0f}',
    'total_rec': '{:,.0f}'
}))

print("\n" + "-" * 30)
print("🛠️ 前十大開發商策略驗證")
print("-" * 30)
display(dev_check.style.format({
    'avg_price': '${:.2f}',
    'avg_discount': '{:.1%}',
    'avg_rec': '{:,.0f}',
    'total_rec': '{:,.0f}'
}))

In [ ]:
# 算活躍遊戲(有推薦數)的折扣
mask = df['discount'] > 0
df_discount = df_paid.loc[mask].copy()
df_no_discount = df_paid.loc[~mask].copy()
df_discount_valid = df_active_paid[
    (df_active_paid['discount(%)'] > 0) &
    (df_active_paid['average_playtime_2weeks'] > 0) &
    (df_active_paid['median_playtime_2weeks'] > 0)
    ].copy()
df_no_discount_valid = df_active_paid[
    (df_active_paid['discount(%)'] == 0) &
    (df_active_paid['average_playtime_2weeks'] > 0) &
    (df_active_paid['median_playtime_2weeks'] > 0)
].copy()
df_valid = df_active_paid[
    (df_active_paid['average_playtime_2weeks'] > 0) &
    (df_active_paid['median_playtime_2weeks'] > 0)
].copy()
df_peak = df_active_paid[df_active_paid['median_playtime_change'] > 1].copy()
avg_discount = df_discount_valid['discount(%)'].mean()
print(f"當前活躍遊戲的平均折扣力道為: {avg_discount:.1%}")

# 比較 Sony 與其他人的折扣力道
sony_discount = df_discount_valid[df_discount_valid['publisher'].str.contains('PlayStation', na=False)]['discount(%)'].mean()
print(f"Sony 遊戲的平均折扣力道: {sony_discount:.1%}")

In [ ]:
# # 找出玩家人數明顯上升的遊戲

cols_to_show = ['name', 'price', 'discount(%)', 'average_playtime_forever', 'median_playtime_change', 'recommendations']
result = df_peak.sort_values('median_playtime_change', ascending=False)[cols_to_show]

result

In [ ]:
# 折扣與遊玩人數統計，加上無折扣組與折扣分群組整合在一張表中。

def discount_performance_full(df_active_paid):
    df = df_active_paid.copy()

    bins_discount = [0, 0.3, 0.5, 0.8, 1.0]
    labels_discount = ['0–30%', '30–50%', '50–80%', '80–100%']

    # 分開「無折扣組」與「有折扣組」
    df_no_dis = df[df['discount(%)'] == 0]
    df_dis = df[df['discount(%)'] > 0].copy()

    df_dis['discount_group'] = pd.cut(
        df_dis['discount(%)'],
        bins=bins_discount,
        labels=labels_discount,
        include_lowest=False
    )

    # 計算有折扣組的統計
    stats_dis = df_dis.groupby('discount_group', observed=True).agg(
        遊戲數量=('appid', 'count'),
        平均折後價=('price', 'mean'),
        平均原價=('original_price', 'mean'),
        遊玩增長中位數=('median_playtime_change', 'median'),
        遊玩增長平均值=('median_playtime_change', 'mean'),
        平均推薦數=('recommendations', 'mean')
    ).reset_index()

    # 計算無折扣組作為基準列
    baseline_row = pd.DataFrame([{
        'discount_group': '無折扣 (基準線)',
        '遊戲數量': len(df_no_dis),
        '平均折後價': df_no_dis['price'].mean(),
        '平均原價': df_no_dis['price'].mean(),
        '遊玩增長中位數': df_no_dis['median_playtime_change'].median(),
        '遊玩增長平均值': df_no_dis['median_playtime_change'].mean(),
        '平均推薦數': df_no_dis['recommendations'].mean()
    }])


    final_report = pd.concat([baseline_row, stats_dis], ignore_index=True)
    return final_report

# df_valid = df_active_paid[
#     (df_active_paid['average_playtime_2weeks'] > 0) &
#     (df_active_paid['median_playtime_2weeks'] > 0)
# ].copy()
discount_performance_full(df_valid)

In [ ]:
df_valid['price_group'] = pd.cut(
    df_valid['original_price'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

df_valid['discount_group'] = pd.cut(
    df_valid['discount(%)'],
    bins=[0, 0.3, 0.5, 0.8, 1.0],
    labels=['0–30%', '30–50%', '50–80%', '80–100%'],
    include_lowest=False
)

heatmap_data = df_valid.groupby(['price_group', 'discount_group'], observed=True).agg(
    平均推薦=('recommendations', 'mean'),
    中位數推薦=('recommendations', 'median'),
    樣本數=('appid', 'count')
).reset_index()

baseline_median = discount_performance_table.loc[
    discount_performance_table['discount_group'] == '無折扣 (基準線)', '遊玩增長中位數'
].values[0]

baseline_mean = discount_performance_table.loc[
    discount_performance_table['discount_group'] == '無折扣 (基準線)', '遊玩增長平均值'
].values[0]

# 2. 計算交叉統計
hm_flow = df_valid.groupby(['price_group', 'discount_group'], observed=True).agg(
    實際中位數=('median_playtime_change', 'median'),
    實際平均值=('median_playtime_change', 'mean')
).reset_index()

# 3. 計算增益 (Lift)
hm_flow['中位數增益'] = hm_flow['實際中位數'] - baseline_median
hm_flow['平均值增益'] = hm_flow['實際平均值'] - baseline_mean

In [ ]:
import plotly.express as px
# 裁剪異常值，確保色階集中在基準線附近的有效範圍
hm_flow['平均值增益_繪圖用'] = hm_flow['平均值增益'].clip(-1, 1)

# 繪圖時 z 軸使用 '平均值增益_繪圖用'
fig_avg = px.density_heatmap(
    hm_flow,
    x="discount_group",
    y="price_group",
    z="平均值增益_繪圖用", # 修改此處
    histfunc="sum",
    text_auto=".2f",
    color_continuous_scale="RdBu",
    color_continuous_midpoint=0,
    title=f"平均值引流增益 (對比基準: {baseline_mean:.2f}) - 已裁剪異常值",
    labels={'price_group': '定價區間', 'discount_group': '折扣帶', '平均值增益_繪圖用': '相對增益'}
)

fig_avg.update_yaxes(categoryorder='array', categoryarray=labels)
fig_avg.show()

In [ ]:
# 只保留活躍遊戲，刪除極端值('median_playtime_change'>20)
df_plot = df_active.copy()
df_plot = df_plot[df_plot['median_playtime_change']<20].copy()

# 建立互動圖，初始顯示 median
fig = px.scatter(
    df_plot,
    x='release_date_dt',
    y='median_playtime_change',
    color=df_plot['Free/paid'].map({0: 'Free', 1: 'Paid'}),
    color_discrete_map={'Free': 'red', 'Paid': 'blue'},  # 固定顏色
    hover_data=['name', 'discount(%)', 'average_playtime_change', 'median_playtime_change'],
    labels={
        'y': 'Playtime Change',
        'release_date_dt': 'Release Date',
        'color': 'Free / Paid'
    },
    title='Playtime Change Over Release Date (Free vs Paid Games)'
)


# 加上按鈕切換 median / average
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(
                    label="Median",
                    method="update",
                    args=[{"y": [df_plot['median_playtime_change']]},
                     {"yaxis": {"title": "Median Playtime Change",
                                "tickformat": ".0%"}}]
                ),
                dict(
                    label="Average",
                    method="update",
                    args=[{"y": [df_plot['average_playtime_change']]},
                     {"yaxis": {"title": "Average Playtime Change",
                                "tickformat": ".0%"}}]
                ),
            ],
            direction="down",
            showactive=True,
        )
    ]
)
fig.update_yaxes(tickformat=".0%")

fig.show()

# 市場分析六：視覺呈現

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
import ipywidgets as widgets
from IPython.display import display
import matplotlib.colors as mcolors
import colorsys

!apt-get update -qq
!apt-get install -y fonts-noto-cjk

def get_colab_cjk_font():
    return "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"

In [ ]:
# 顏色字典（RGB 已轉 0~1，可直接用於 WordCloud / MDS / Matplotlib）
FIXED_GENRE_COLORS = {
    # ===== genres（主色，高辨識） =====
    'genres_Indie': (229/255, 57/255, 53/255),
    'genres_Adventure': (251/255, 140/255, 0/255),
    'genres_Action': (253/255, 216/255, 53/255),
    'genres_Casual': (30/255, 136/255, 229/255),
    'genres_Simulation': (67/255, 160/255, 71/255),
    'genres_RPG': (38/255, 198/255, 218/255),
    'genres_Strategy': (142/255, 36/255, 170/255),
    'genres_Early Access': (117/255, 117/255, 117/255),

    # ===== categories（功能 / 平台語意，之後可淡化） =====
    # 高頻・Steam 功能
    'categories_Family Sharing': (25/255, 118/255, 210/255),
    'categories_Steam Achievements': (2/255, 136/255, 209/255),
    'categories_Steam Cloud': (38/255, 166/255, 154/255),
    'categories_Steam Trading Cards': (92/255, 107/255, 192/255),

    # 遊玩模式
    'categories_Single-player': (123/255, 31/255, 162/255),
    'categories_Multi-player': (244/255, 81/255, 30/255),
    'categories_Co-op': (251/255, 140/255, 0/255),
    'categories_Online Co-op': (239/255, 108/255, 0/255),
    'categories_PvP': (198/255, 40/255, 40/255),
    'categories_Online PvP': (173/255, 20/255, 87/255),

    # 操作 / UX / 輔助
    'categories_Full controller support': (46/255, 125/255, 50/255),
    'categories_Partial Controller Support': (102/255, 187/255, 106/255),
    'categories_Custom Volume Controls': (129/255, 199/255, 132/255),
    'categories_Playable without Timed Input': (156/255, 204/255, 101/255)
}

In [ ]:
# 6-1. 標籤組合規律
tag_cols = [c for c in df.columns if c.startswith('genres_') or c.startswith('categories_')]

In [ ]:
# @title
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import rcParams

def show_tag_heatmap(data, tag_cols=tags, title=None, top_n=40, figsize=(16,14)):
    font_prop = get_colab_cjk_font()
    tags = [c for c in df_active.columns if c.startswith('genres_') or c.startswith('categories_')]
    # 篩選有效欄位（至少有兩種不同值）
    active = [c for c in tag_cols if data[c].nunique() > 1]

    # 計算標籤出現次數，取 top_n
    top_tags = data[active].sum().sort_values(ascending=False).head(top_n).index

    # 計算相關性矩陣
    corr = data[top_tags].corr()

    # 標籤簡化
    labels = [c.replace('genres_', '').replace('categories_', '') for c in top_tags]

    # 畫圖
    plt.figure(figsize=figsize)
    mask = np.triu(np.ones_like(corr, dtype=bool))  # 上三角遮罩
    sns.heatmap(
        corr,
        mask=mask,
        cmap=sns.diverging_palette(230, 20, as_cmap=True),
        vmax=1.0, center=0, vmin=-1.0,
        square=True, linewidths=.5,
        xticklabels=labels, yticklabels=labels
    )
    plt.title(title, fontsize=20)
    plt.tight_layout()
    plt.show()

show_tag_heatmap(df_outliers)

In [1]:
# 對genre和category一對一配對，計算資料集中的出現次數
# 出現次數越多代表該類型遊戲最常有關聯性的標籤，以面積表示


# 淡化顏色（只改 alpha）
def fade_color(color, alpha=0.5):
    r, g, b = color[:3]
    return (r, g, b, alpha)

# 互動式 MDS 函式（支援 top N 標籤 + recommendations 閾值 + categories 淡化）
def generate_mds_interactive(
    data_subset,
    top_n_tags=40,
    noise_threshold=0.05,
    fade_alpha=0.8
):
    # 🔹 只取 genres_ 和 categories_ 開頭的標籤
    tag_cols = [c for c in data_subset.columns if c.startswith('genres_') or c.startswith('categories_')]

    # 顏色函式
    def get_color(tag):
        if tag in FIXED_GENRE_COLORS:
            color = FIXED_GENRE_COLORS[tag]
            if tag.startswith('categories_'):
                color = fade_color(color, alpha=fade_alpha)
            return color
        else:
            return (0.5,0.5,0.5,0.7)  # fallback 灰色

    # 繪圖函式
    def plot_mds(top_n_tags_slider, recommendations_threshold):
        # 🔹 只取 recommendations 大於閾值的資料
        if 'recommendations' in data_subset.columns:
            df_filtered = data_subset[data_subset['recommendations'] > recommendations_threshold]
        else:
            df_filtered = data_subset.copy()

        # 🔹 取 top N 標籤
        tag_counts = df_filtered[tag_cols].sum().sort_values(ascending=False).head(top_n_tags_slider)
        target_tags = tag_counts.index.tolist()

        # 🔹 計算相關矩陣
        corr_matrix = df_filtered[target_tags].corr().fillna(0)
        corr_matrix[corr_matrix.abs() < noise_threshold] = 0
        distance_matrix = 1 - np.abs(corr_matrix.values)

        # 🔹 MDS
        mds = MDS(
            n_components=2,
            dissimilarity="precomputed",
            random_state=42,
            n_init=4,
            max_iter=300,
            normalized_stress="auto"
        )
        pos = mds.fit_transform(distance_matrix)

        # 🔹 顏色
        final_colors = [get_color(tag) for tag in target_tags]

        # 🔹 繪圖
        plt.figure(figsize=(16,12))
        sizes = (tag_counts.values / tag_counts.values.max()) * 20000
        plt.scatter(pos[:,0], pos[:,1], s=sizes, c=final_colors, alpha=0.7, edgecolors='white', linewidth=1)

        for i, tag in enumerate(target_tags):
            label = tag.replace('genres_', '').replace('categories_', '')
            f_size = max(10, min(18, int(tag_counts.values[i] / tag_counts.values.max() * 22)))
            plt.annotate(label, (pos[i,0], pos[i,1]), ha='center', va='center',
                         fontsize=f_size,
                         fontweight='bold' if tag in FIXED_GENRE_COLORS else 'normal')

        plt.title(f"Interactive MDS Tag Map\nTop {top_n_tags_slider} Tags | recommendations > {recommendations_threshold}", fontsize=22, pad=30)
        plt.grid(True, linestyle='--', alpha=0.15)
        plt.show()

    # 🔹 Slider 設定
    slider_top_n = widgets.IntSlider(
        value=top_n_tags, min=5, max=len(tag_cols), step=1,
        description='對出現率最高的N個標籤計算關聯性:',
        layout=widgets.Layout(width='700px'),
        style={'description_width': '350px'}
    )
    slider_recs = widgets.IntSlider(
        value=0, min=0, max=int(data_subset['recommendations'].max()), step=100,
        description='顯示 recommendations 大於:',
        layout=widgets.Layout(width='700px'),
        style={'description_width': '350px'}
    )
    # 🔹 排列
    ui = widgets.VBox([slider_top_n, slider_recs])
    widgets.interact(plot_mds, top_n_tags_slider=slider_top_n, recommendations_threshold=slider_recs)

generate_mds_interactive(df)

In [ ]:
# @title
# 中文映射字典
genres_map = {
    # genres
    'genres_Accounting': '會計',
    'genres_Action': '動作',
    'genres_Adventure': '冒險',
    'genres_Animation & Modeling': '動畫建模',
    'genres_Audio Production': '音效',
    'genres_Casual': '休閒',
    'genres_Design & Illustration': '設計與插畫',
    'genres_Early Access': '搶先體驗',
    'genres_Education': '教育',
    'genres_Free To Play': '免費遊玩',
    'genres_Game Development': '遊戲開發',
    'genres_Indie': '獨立',
    'genres_Massively Multiplayer': '大型多人',
    'genres_Photo Editing': '拍照',
    'genres_RPG': '角色扮演',
    'genres_Racing': '競速',
    'genres_Simulation': '模擬',
    'genres_Software Training': '軟體教育',
    'genres_Sports': '運動',
    'genres_Strategy': '策略',
    'genres_Utilities': '系統工具',
    'genres_Video Production': '影片製作',
    'genres_Violent': '暴力',
    'genres_Web Publishing': '相關網頁',

    # categories
    'categories_Adjustable Difficulty': '難度調整',
    'categories_Adjustable Text Size': '文字調整',
    'categories_Camera Comfort': '鏡頭舒適',
    'categories_Captions available': '字幕',
    'categories_Chat Speech-to-text': '語音→文字',
    'categories_Chat Text-to-speech': '文字→語音',
    'categories_Co-op': '合作模式',
    'categories_Color Alternatives': '色盲替代',
    'categories_Commentary available': '評論',
    'categories_Cross-Platform Multiplayer': '跨平台多人',
    'categories_Custom Volume Controls': '音量控制',
    'categories_Family Sharing': '家庭共享',
    'categories_Full controller support': '全手把',
    'categories_HDR available': 'HDR',
    'categories_In-App Purchases': '內購',
    'categories_Includes Source SDK': '開發工具',
    'categories_Includes level editor': '關卡編輯',
    'categories_Keyboard Only Option': '鍵盤模式',
    'categories_LAN Co-op': '連線合作',
    'categories_LAN PvP': '連線對戰',
    'categories_MMO': '線上遊戲',
    'categories_Mouse Only Option': '滑鼠模式',
    'categories_Multi-player': '多人',
    'categories_Narrated Game Menus': '選單語音',
    'categories_Online Co-op': '線上合作',
    'categories_Online PvP': '線上對戰',
    'categories_Partial Controller Support': '部分手把',
    'categories_Playable without Timed Input': '關閉QTE',
    'categories_PvP': '玩家對戰',
    'categories_Remote Play Together': '遠端',
    'categories_Remote Play on Phone': '遠端手機',
    'categories_Remote Play on TV': '遠端電視',
    'categories_Remote Play on Tablet': '遠端平板',
    'categories_Save Anytime': '隨時存檔',
    'categories_Shared/Split Screen': '分屏',
    'categories_Shared/Split Screen Co-op': '分屏合作',
    'categories_Shared/Split Screen PvP': '分屏對戰',
    'categories_Single-player': '單人',
    'categories_Stats': '統計數據',
    'categories_Steam Achievements': 'Steam 成就',
    'categories_Steam Cloud': 'Steam 雲端',
    'categories_Steam Leaderboards': 'Steam 排行榜',
    'categories_Steam Timeline': 'Steam 時間軸',
    'categories_Steam Trading Cards': 'Steam 集卡',
    'categories_Steam Turn Notifications': 'Steam 回合通知',
    'categories_Steam Workshop': 'Steam 工坊',
    'categories_SteamVR Collectibles': 'Steam VR 收藏品',
    'categories_Stereo Sound': '立體聲',
    'categories_Subtitle Options': '字幕選擇',
    'categories_Surround Sound': '環繞音效',
    'categories_Touch Only Option': '僅觸控',
    'categories_Tracked Controller Support': 'VR手把支援',
    'categories_VR Only': '僅 VR',
    'categories_VR Support': '支援 VR',
    'categories_VR Supported': 'VR 支援',
    'categories_Valve Anti-Cheat enabled': '啟用 Valve 反作弊'
}

In [ ]:
# 文字雲圖
# 只計算單個標籤的出現次數，無關聯性

# @title
def generate_wordcloud_genre_category(
    df_wordcloud,
    mapping=None,
    color_dict=None,
    center_text=None,
    font_path=None,
    figsize=(12,6),
    bg_color='white'
):
    from wordcloud import WordCloud
    import matplotlib.pyplot as plt
    from matplotlib.font_manager import FontProperties
    import random

    if font_path is None:
        font_path = get_colab_cjk_font()

    df_wc = df_wordcloud.copy()  # 取全部

    cols = [
        c for c in df_wc.columns
        if (c.startswith("genres_") or c.startswith("categories_"))
        and mapping and c in mapping
    ]

    # 計數
    freq = {}
    for c in cols:
        cnt = int(df_wc[c].sum())
        if cnt > 0:
            freq[mapping[c]] = cnt
    color_dict = {
    # ===== genres（可微調但保持主色） =====
    'genres_Indie': '#E53935',                 # 紅
    'genres_Adventure': '#FB8C00',             # 橘
    'genres_Action': '#FDD835',                # 黃
    'genres_Casual': '#1E88E5',                 # 藍
    'genres_Simulation': '#43A047',             # 綠
    'genres_RPG': '#26C6DA',                    # 青
    'genres_Strategy': '#8E24AA',               # 紫
    'genres_Early Access': '#757575',           # 灰

    # ===== categories（依功能語意） =====
    # 高頻・平台 / Steam 功能
    'categories_Family Sharing': '#1976D2',     # 深藍
    'categories_Steam Achievements': '#0288D1', # 藍
    'categories_Steam Cloud': '#26A69A',         # 青綠
    'categories_Steam Trading Cards': '#5C6BC0',# 藍紫

    # 遊玩模式
    'categories_Single-player': '#7B1FA2',      # 紫
    'categories_Multi-player': '#F4511E',       # 橘紅
    'categories_Co-op': '#FB8C00',               # 橘
    'categories_Online Co-op': '#EF6C00',        # 深橘
    'categories_PvP': '#C62828',                 # 深紅
    'categories_Online PvP': '#AD1457',          # 紅紫

    # 操作 / 輔助 / UX
    'categories_Full controller support': '#2E7D32',   # 深綠
    'categories_Partial Controller Support': '#66BB6A',# 淺綠
    'categories_Custom Volume Controls': '#81C784',    # 柔綠
    'categories_Playable without Timed Input': '#9CCC65'# 黃綠
    }
    def color_func(word, **kwargs):
        import matplotlib.colors as mcolors
        # 迭代 mapping 找對應單字
        for k, v in mapping.items():
            if v == word and color_dict and k in color_dict:
                hex_color = color_dict[k]  # Hex 字串 '#RRGGBB'
                # 轉 RGB tuple (0~255)
                r, g, b = [int(x*255) if isinstance(x,float) else int(x) for x in mcolors.to_rgb(hex_color)]

                # categories_ 類別淡化 50%
                if k.startswith("categories_"):
                    r = int(r + (255 - r) * 0.5)
                    g = int(g + (255 - g) * 0.5)
                    b = int(b + (255 - b) * 0.5)

                return f"rgb({r},{g},{b})"  # WordCloud 可接受

        # fallback 灰色
        g = random.randint(80, 180)
        return f"rgb({g},{g},{g})"


    wc = WordCloud(
        width=900,
        height=450,
        background_color=bg_color,
        font_path=font_path,
        collocations=False
    ).generate_from_frequencies(freq)

    plt.figure(figsize=figsize)
    plt.imshow(wc.recolor(color_func=color_func))
    plt.axis("off")

    if center_text is not None:
        plt.text(
            wc.width / 2,
            wc.height / 2,
            center_text,
            fontsize=80,
            fontproperties=get_colab_cjk_font(),  # font_prop 一定有定義
            weight="bold",
            ha="center",
            va="center",
            alpha=0.6
        )

    plt.show()
    return wc


In [ ]:
# 呼叫文字雲
df_wordcloud = df

wc = generate_wordcloud_genre_category(
    df_wordcloud,
    mapping=genres_map,
    color_dict=FIXED_GENRE_COLORS,
    center_text=None
)

In [ ]:
# @title
import ipywidgets as widgets
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random

def generate_wordcloud_interactive(
    datasets_dict,        # { '按鈕文字': dataframe }
    mapping,              # { col_name: 顯示文字 }
    initial_dataset_name=None,
    font_path=None,
    figsize=(12,6),
    bg_color='white',
    fade_categories_alpha=0.5
):
    if font_path is None:
        font_path = get_colab_cjk_font()

    # ===== 內含色表（整數 RGB 0~255） =====
    FIXED_GENRE_COLORS = {
        # ===== genres（主色，高辨識） =====
        'genres_Indie': (229, 57, 53),
        'genres_Adventure': (251, 140, 0),
        'genres_Action': (253, 216, 53),
        'genres_Casual': (30, 136, 229),
        'genres_Simulation': (67, 160, 71),
        'genres_RPG': (38, 198, 218),
        'genres_Strategy': (142, 36, 170),
        'genres_Early Access': (117, 117, 117),

        # ===== categories（功能 / 平台 / UX） =====
        'categories_Family Sharing': (25, 118, 210),
        'categories_Steam Achievements': (2, 136, 209),
        'categories_Steam Cloud': (38, 166, 154),
        'categories_Steam Trading Cards': (92, 107, 192),

        'categories_Single-player': (123, 31, 162),
        'categories_Multi-player': (244, 81, 30),
        'categories_Co-op': (251, 140, 0),
        'categories_Online Co-op': (239, 108, 0),
        'categories_PvP': (198, 40, 40),
        'categories_Online PvP': (173, 20, 87),

        'categories_Full controller support': (46, 125, 50),
        'categories_Partial Controller Support': (102, 187, 106),
        'categories_Custom Volume Controls': (129, 199, 132),
        'categories_Playable without Timed Input': (156, 204, 101)
    }

    output = widgets.Output()

    def draw_wordcloud(name):
        df_wc = datasets_dict[name]
        cols = [
            c for c in df_wc.columns
            if (c.startswith("genres_") or c.startswith("categories_"))
            and mapping and c in mapping
        ]

        # 計數
        freq = {}
        for c in cols:
            cnt = int(df_wc[c].sum())
            if cnt > 0:
                freq[mapping[c]] = cnt

        # 顏色函式
        def color_func(word, **kwargs):
            for k, v in mapping.items():
                if v == word and k in FIXED_GENRE_COLORS:
                    r, g, b = FIXED_GENRE_COLORS[k]
                    if k.startswith("categories_"):
                        # categories_ 淡化
                        r = int(r + (255 - r) * fade_categories_alpha)
                        g = int(g + (255 - g) * fade_categories_alpha)
                        b = int(b + (255 - b) * fade_categories_alpha)
                        return f"rgb({r},{g},{b})"
                    return f"rgb({r},{g},{b})"
            # fallback 灰
            g_val = random.randint(80, 180)
            return f"rgb({g_val},{g_val},{g_val})"

        wc = WordCloud(
            width=900,
            height=450,
            background_color=bg_color,
            font_path=font_path,
            collocations=False
        ).generate_from_frequencies(freq)

        with output:
            output.clear_output(wait=True)
            plt.figure(figsize=figsize)
            plt.imshow(wc.recolor(color_func=color_func))
            plt.axis("off")
            plt.show()

    # 生成按鈕
    buttons = []
    for name in datasets_dict.keys():
        b = widgets.Button(description=name)
        b.on_click(lambda btn, n=name: draw_wordcloud(n))
        buttons.append(b)

    display(widgets.HBox(buttons))
    display(output)

    # 初始顯示
    if initial_dataset_name is not None:
        draw_wordcloud(initial_dataset_name)


In [ ]:
# 把文字雲做成可以按照分類切換的樣式
datasets_dict = {
    "全部遊戲": df,
    "推薦遊戲": df_high_pop,
    "獨立遊戲": df_indie,
    "免費大作": df_free_hpop,
    "超級熱門": df_outliers,
    "得獎作品": df_existing_awards
}

generate_wordcloud_interactive(
    datasets_dict=datasets_dict,
    mapping=genres_map,
    initial_dataset_name="全部遊戲"
)

# 結束